In [127]:
from robobrowser import RoboBrowser
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

#Open URL and create RoboBrowser Instance
browser = RoboBrowser(parser='html.parser', history=True)
dates = pd.date_range('2016-01-01', periods=2, freq='W').format('YYYY-MM-dd')[1:]
#print(dates)
results = pd.DataFrame()
for d in dates:
    url='https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/'+ str(d) +'/DE/24'
    browser.open(url)  
    soup = bs(str(browser.parsed), 'html.parser')
    weekData = soup.find("table", class_="hours")
    #print(weekData)
    dates = pd.date_range(pd.to_datetime(d) - pd.to_timedelta(6, 'D'), periods=7, freq="D")
    hours = [h.get_text().strip() for h in weekData.select(".no-border .title")]
    priceRows = weekData.findAll("tr", class_="no-border")
    prices = np.zeros((24, 7))
    for i in range(24):
        prices[i] = [float(p.get_text().replace(",",".")) for p in priceRows[i].findAll("td")[2:]]
    df = pd.DataFrame(prices)
    df.index = hours
    df.columns = dates
    df = df.transpose()
    # cut off last dates of 2015
    if d == "2016-01-03":
        df = df[4:]
    #print(dates)
    #print(hours)
    #print(prices)
 
    #print(df)
    if results.empty:
        results = df
    else:
        results = results.append(df)
print(results)


['2016-01-03', '2016-01-10']
            00 - 01  01 - 02  02 - 03  03 - 04  04 - 05  05 - 06  06 - 07  \
2016-01-01    23.86    22.39    20.59    16.81    17.41    17.02    15.86   
2016-01-02    22.20    16.57    15.35    12.77    11.27    11.91    12.62   
2016-01-03    19.65    18.91    14.64    10.12     7.14     9.41     4.14   
2016-01-04    13.78    13.44    11.46     5.55     5.00    10.05    19.11   
2016-01-05    25.03    24.42    22.34    21.50    20.00    23.53    28.96   
2016-01-06    26.62    25.58    24.59    22.50    23.16    24.99    26.62   
2016-01-07    26.41    26.05    24.93    22.39    22.32    24.20    31.00   
2016-01-08    24.36    19.33    16.10    15.00    15.00    18.14    25.21   
2016-01-09    31.11    29.00    24.30    23.23    21.96    22.45    23.07   
2016-01-10    15.58    19.05    16.96    12.75    10.81    10.85     9.85   

            07 - 08  08 - 09  09 - 10   ...     14 - 15  15 - 16  16 - 17  \
2016-01-01    18.16    17.73    19.77   ...   